# Column Sense Group - Full Code for Project
**Column Sense Members: Omar Alsuhaibani, Varshini Batti, Nilay Bhatt, Tom Shin**

*BIS 638 (Fall 2024) - Dr. Kei Hoi Cheung, Dr. George Hauser*

In [ ]:
#Import necessary libraries
import sqlite3
import gzip
import pandas as pd
import os
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import numpy as np

In [ ]:
#Database path and connection settings
db_path = '/path/to/your/db/mimiciv.db'
conn = sqlite3.connect(db_path)

In [ ]:
            # You need to do the progress of processing the files and loading them to dataframes and such by installing them in the preferred way from the PhysioNet database #

**Filtering MIMIC IV tables for VAP patients**

In [ ]:
#Filtering MIMIC IV tables to only subjects of interest (ventilator-associated pneumonia (VAP) patients):
output_csv_path = '//vap_patients.csv'
vap_patients = """
WITH VAP_PATIENTS AS (
  SELECT 
    a.subject_id, 
    a.hadm_id, 
    di.icd_code, 
    di.icd_version, 
    di.long_title
  FROM admissions AS a
  JOIN diagnoses_icd AS d
    ON icu.hadm_id = d.hadm_id
  JOIN d_icd_diagnoses AS di
    ON d.icd_code = di.icd_code AND d.icd_version = di.icd_version
  WHERE di.long_title LIKE '%ventilator-associated pneumonia%'
     OR (di.icd_version = 9 AND di.icd_code = '99731')  -- ICD-9 code
     OR (di.icd_version = 10 AND di.icd_code = 'J95851')  -- ICD-10 code
)
SELECT * FROM VAP_PATIENTS;
"""
conn = sqlite3.connect(db_path)
try:
    #Save as a DataFrame:
    vap_patients_df = pd.read_sql_query(vap_patients, conn)
    print("Count of Patients Diagnosed with VAP:")
    print(vap_patients_df.subject_id.nunique())
    
    #Save as a table in DB:
    vap_patients_df.to_sql('VAP_PATIENTS', conn, if_exists='replace', index=False)
    print("Table created successfully.")
    
    #Export table to CSV file:
    vap_patients_df.to_csv(output_csv_path, index=False)
    print(f"Exported VAP_PATIENTStable to {output_csv_path}")
except Exception as e:
    print(f"Error exporting VAP_PATIENTStable: {e}")
finally:
    conn.close()

**Now that we filtered the database for VAP Patients, we get more necessary information that can be utilized**:

In [ ]:
# Helper function to execute queries
def execute_query(query):
    try:
        result = pd.read_sql_query(query, conn)
        print(f"Query executed successfully. Preview:\n{result.head()}")
        return result
    except Exception as e:
        print(f"Error executing query: {e}")
        return None

# Demographic data
demographics_query = """
SELECT 
    vp.subject_id,
    vp.hadm_id,
    -- Calculate admission age
    p.anchor_age + 
    CAST((strftime('%Y', a.admittime) - p.anchor_year) AS INTEGER) AS admission_age,
    p.gender,
    a.race AS ethnicity
FROM VAP_PATIENTS vp
JOIN patients p ON vp.subject_id = p.subject_id
JOIN admissions a ON vp.hadm_id = a.hadm_id;
"""

# Labs and measurements
labs_query = """
SELECT 
    vp.subject_id,
    li.label AS lab_test,
    le.valuenum AS value,
    le.charttime
FROM VAP_PATIENTS vp
JOIN labevents le ON vp.hadm_id = le.hadm_id
JOIN d_labitems li ON le.itemid = li.itemid
WHERE li.label IN ('WBC', 'CRP', 'Procalcitonin');
"""

# Outcomes
outcomes_query = """
SELECT 
    vp.subject_id,
    a.hospital_expire_flag AS mortality,
    icu.outtime IS NOT NULL AS icu_discharge
FROM VAP_PATIENTS vp
JOIN admissions a ON vp.hadm_id = a.hadm_id
LEFT JOIN icustays icu ON vp.stay_id = icu.stay_id;
"""

# Execute queries and store results in DataFrames
demographics = execute_query(demographics_query)
labs = execute_query(labs_query)
outcomes = execute_query(outcomes_query)

# Combine all data into one table
try:
    unified_table = demographics \
        .merge(labs, on='subject_id', how='left') \
        .merge(outcomes, on='subject_id', how='left')
    
    print("Unified table created successfully. Preview:")
    print(unified_table.head())
    
    # Save to database or CSV
    unified_table.to_sql('unified_vap_table', conn, if_exists='replace', index=False)
    unified_table.to_csv('unified_vap_table.csv', index=False)
    print("Unified table saved to database and CSV.")
except Exception as e:
    print(f"Error creating unified table: {e}")

**To filter and find the treatments administered to VAP patients, we did some research and found that antibiotics and vasoactive agents are the most relative classes of treatments to look into.**

We then utilized resources found on the official MIMIC GitHub page, and we filtered it to the VAP patients:

Antibiotics:

In [ ]:
query = """
WITH abx_VAP AS (
    SELECT DISTINCT
        drug, route,
        CASE
            WHEN LOWER(drug) LIKE '%adoxa%' THEN 1
            WHEN LOWER(drug) LIKE '%ala-tet%' THEN 1
            WHEN LOWER(drug) LIKE '%alodox%' THEN 1
            WHEN LOWER(drug) LIKE '%amikacin%' THEN 1
            WHEN LOWER(drug) LIKE '%amikin%' THEN 1
            WHEN LOWER(drug) LIKE '%amoxicill%' THEN 1
            WHEN LOWER(drug) LIKE '%amphotericin%' THEN 1
            WHEN LOWER(drug) LIKE '%anidulafungin%' THEN 1
            WHEN LOWER(drug) LIKE '%ancef%' THEN 1
            WHEN LOWER(drug) LIKE '%clavulanate%' THEN 1
            WHEN LOWER(drug) LIKE '%ampicillin%' THEN 1
            WHEN LOWER(drug) LIKE '%augmentin%' THEN 1
            WHEN LOWER(drug) LIKE '%avelox%' THEN 1
            WHEN LOWER(drug) LIKE '%avidoxy%' THEN 1
            WHEN LOWER(drug) LIKE '%azactam%' THEN 1
            WHEN LOWER(drug) LIKE '%azithromycin%' THEN 1
            WHEN LOWER(drug) LIKE '%aztreonam%' THEN 1
            WHEN LOWER(drug) LIKE '%axetil%' THEN 1
            WHEN LOWER(drug) LIKE '%bactocill%' THEN 1
            WHEN LOWER(drug) LIKE '%bactrim%' THEN 1
            WHEN LOWER(drug) LIKE '%bactroban%' THEN 1
            WHEN LOWER(drug) LIKE '%bethkis%' THEN 1
            WHEN LOWER(drug) LIKE '%biaxin%' THEN 1
            WHEN LOWER(drug) LIKE '%bicillin l-a%' THEN 1
            WHEN LOWER(drug) LIKE '%cayston%' THEN 1
            WHEN LOWER(drug) LIKE '%cefazolin%' THEN 1
            WHEN LOWER(drug) LIKE '%cedax%' THEN 1
            WHEN LOWER(drug) LIKE '%cefoxitin%' THEN 1
            WHEN LOWER(drug) LIKE '%ceftazidime%' THEN 1
            WHEN LOWER(drug) LIKE '%cefaclor%' THEN 1
            WHEN LOWER(drug) LIKE '%cefadroxil%' THEN 1
            WHEN LOWER(drug) LIKE '%cefdinir%' THEN 1
            WHEN LOWER(drug) LIKE '%cefditoren%' THEN 1
            WHEN LOWER(drug) LIKE '%cefepime%' THEN 1
            WHEN LOWER(drug) LIKE '%cefotan%' THEN 1
            WHEN LOWER(drug) LIKE '%cefotetan%' THEN 1
            WHEN LOWER(drug) LIKE '%cefotaxime%' THEN 1
            WHEN LOWER(drug) LIKE '%ceftaroline%' THEN 1
            WHEN LOWER(drug) LIKE '%cefpodoxime%' THEN 1
            WHEN LOWER(drug) LIKE '%cefpirome%' THEN 1
            WHEN LOWER(drug) LIKE '%cefprozil%' THEN 1
            WHEN LOWER(drug) LIKE '%ceftibuten%' THEN 1
            WHEN LOWER(drug) LIKE '%ceftin%' THEN 1
            WHEN LOWER(drug) LIKE '%ceftriaxone%' THEN 1
            WHEN LOWER(drug) LIKE '%cefuroxime%' THEN 1
            WHEN LOWER(drug) LIKE '%cephalexin%' THEN 1
            WHEN LOWER(drug) LIKE '%cephalothin%' THEN 1
            WHEN LOWER(drug) LIKE '%cephapririn%' THEN 1
            WHEN LOWER(drug) LIKE '%chloramphenicol%' THEN 1
            WHEN LOWER(drug) LIKE '%cipro%' THEN 1
            WHEN LOWER(drug) LIKE '%ciprofloxacin%' THEN 1
            WHEN LOWER(drug) LIKE '%claforan%' THEN 1
            WHEN LOWER(drug) LIKE '%clarithromycin%' THEN 1
            WHEN LOWER(drug) LIKE '%cleocin%' THEN 1
            WHEN LOWER(drug) LIKE '%clindamycin%' THEN 1
            WHEN LOWER(drug) LIKE '%cubicin%' THEN 1
            WHEN LOWER(drug) LIKE '%dicloxacillin%' THEN 1
            WHEN LOWER(drug) LIKE '%dirithromycin%' THEN 1
            WHEN LOWER(drug) LIKE '%doryx%' THEN 1
            WHEN LOWER(drug) LIKE '%doxycy%' THEN 1
            WHEN LOWER(drug) LIKE '%duricef%' THEN 1
            WHEN LOWER(drug) LIKE '%dynacin%' THEN 1
            WHEN LOWER(drug) LIKE '%ery-tab%' THEN 1
            WHEN LOWER(drug) LIKE '%eryped%' THEN 1
            WHEN LOWER(drug) LIKE '%eryc%' THEN 1
            WHEN LOWER(drug) LIKE '%erythrocin%' THEN 1
            WHEN LOWER(drug) LIKE '%erythromycin%' THEN 1
            WHEN LOWER(drug) LIKE '%factive%' THEN 1
            WHEN LOWER(drug) LIKE '%flagyl%' THEN 1
            WHEN LOWER(drug) LIKE '%fortaz%' THEN 1
            WHEN LOWER(drug) LIKE '%furadantin%' THEN 1
            WHEN LOWER(drug) LIKE '%garamycin%' THEN 1
            WHEN LOWER(drug) LIKE '%gentamicin%' THEN 1
            WHEN LOWER(drug) LIKE '%kanamycin%' THEN 1
            WHEN LOWER(drug) LIKE '%keflex%' THEN 1
            WHEN LOWER(drug) LIKE '%kefzol%' THEN 1
            WHEN LOWER(drug) LIKE '%ketek%' THEN 1
            WHEN LOWER(drug) LIKE '%levaquin%' THEN 1
            WHEN LOWER(drug) LIKE '%levofloxacin%' THEN 1
            WHEN LOWER(drug) LIKE '%lincocin%' THEN 1
            WHEN LOWER(drug) LIKE '%linezolid%' THEN 1
            WHEN LOWER(drug) LIKE '%macrobid%' THEN 1
            WHEN LOWER(drug) LIKE '%macrodantin%' THEN 1
            WHEN LOWER(drug) LIKE '%maxipime%' THEN 1
            WHEN LOWER(drug) LIKE '%mefoxin%' THEN 1
            WHEN LOWER(drug) LIKE '%metronidazole%' THEN 1
            WHEN LOWER(drug) LIKE '%meropenem%' THEN 1
            WHEN LOWER(drug) LIKE '%methicillin%' THEN 1
            WHEN LOWER(drug) LIKE '%minocin%' THEN 1
            WHEN LOWER(drug) LIKE '%minocycline%' THEN 1
            WHEN LOWER(drug) LIKE '%monodox%' THEN 1
            WHEN LOWER(drug) LIKE '%monurol%' THEN 1
            WHEN LOWER(drug) LIKE '%morgidox%' THEN 1
            WHEN LOWER(drug) LIKE '%moxatag%' THEN 1
            WHEN LOWER(drug) LIKE '%moxifloxacin%' THEN 1
            WHEN LOWER(drug) LIKE '%mupirocin%' THEN 1
            WHEN LOWER(drug) LIKE '%myrac%' THEN 1
            WHEN LOWER(drug) LIKE '%nafcillin%' THEN 1
            WHEN LOWER(drug) LIKE '%neomycin%' THEN 1
            WHEN LOWER(drug) LIKE '%nicazel doxy 30%' THEN 1
            WHEN LOWER(drug) LIKE '%nitrofurantoin%' THEN 1
            WHEN LOWER(drug) LIKE '%norfloxacin%' THEN 1
            WHEN LOWER(drug) LIKE '%noroxin%' THEN 1
            WHEN LOWER(drug) LIKE '%ocudox%' THEN 1
            WHEN LOWER(drug) LIKE '%ofloxacin%' THEN 1
            WHEN LOWER(drug) LIKE '%omnicef%' THEN 1
            WHEN LOWER(drug) LIKE '%oracea%' THEN 1
            WHEN LOWER(drug) LIKE '%oraxyl%' THEN 1
            WHEN LOWER(drug) LIKE '%oxacillin%' THEN 1
            WHEN LOWER(drug) LIKE '%pc pen vk%' THEN 1
            WHEN LOWER(drug) LIKE '%pce dispertab%' THEN 1
            WHEN LOWER(drug) LIKE '%panixine%' THEN 1
            WHEN LOWER(drug) LIKE '%pediazole%' THEN 1
            WHEN LOWER(drug) LIKE '%penicillin%' THEN 1
            WHEN LOWER(drug) LIKE '%periostat%' THEN 1
            WHEN LOWER(drug) LIKE '%pfizerpen%' THEN 1
            WHEN LOWER(drug) LIKE '%piperacillin%' THEN 1
            WHEN LOWER(drug) LIKE '%tazobactam%' THEN 1
            WHEN LOWER(drug) LIKE '%primsol%' THEN 1
            WHEN LOWER(drug) LIKE '%proquin%' THEN 1
            WHEN LOWER(drug) LIKE '%raniclor%' THEN 1
            WHEN LOWER(drug) LIKE '%rifadin%' THEN 1
            WHEN LOWER(drug) LIKE '%rifampin%' THEN 1
            WHEN LOWER(drug) LIKE '%rocephin%' THEN 1
            WHEN LOWER(drug) LIKE '%smz-tmp%' THEN 1
            WHEN LOWER(drug) LIKE '%septra%' THEN 1
            WHEN LOWER(drug) LIKE '%septra ds%' THEN 1
            WHEN LOWER(drug) LIKE '%septra%' THEN 1
            WHEN LOWER(drug) LIKE '%solodyn%' THEN 1
            WHEN LOWER(drug) LIKE '%spectracef%' THEN 1
            WHEN LOWER(drug) LIKE '%streptomycin%' THEN 1
            WHEN LOWER(drug) LIKE '%sulfadiazine%' THEN 1
            WHEN LOWER(drug) LIKE '%sulfamethoxazole%' THEN 1
            WHEN LOWER(drug) LIKE '%trimethoprim%' THEN 1
            WHEN LOWER(drug) LIKE '%sulfatrim%' THEN 1
            WHEN LOWER(drug) LIKE '%sulfisoxazole%' THEN 1
            WHEN LOWER(drug) LIKE '%suprax%' THEN 1
            WHEN LOWER(drug) LIKE '%synercid%' THEN 1
            WHEN LOWER(drug) LIKE '%tazicef%' THEN 1
            WHEN LOWER(drug) LIKE '%tetracycline%' THEN 1
            WHEN LOWER(drug) LIKE '%timentin%' THEN 1
            WHEN LOWER(drug) LIKE '%tobramycin%' THEN 1
            WHEN LOWER(drug) LIKE '%trimethoprim%' THEN 1
            WHEN LOWER(drug) LIKE '%unasyn%' THEN 1
            WHEN LOWER(drug) LIKE '%vancocin%' THEN 1
            WHEN LOWER(drug) LIKE '%vancomycin%' THEN 1
            WHEN LOWER(drug) LIKE '%vantin%' THEN 1
            WHEN LOWER(drug) LIKE '%vibativ%' THEN 1
            WHEN LOWER(drug) LIKE '%vibra-tabs%' THEN 1
            WHEN LOWER(drug) LIKE '%vibramycin%' THEN 1
            WHEN LOWER(drug) LIKE '%zinacef%' THEN 1
            WHEN LOWER(drug) LIKE '%zithromax%' THEN 1
            WHEN LOWER(drug) LIKE '%zosyn%' THEN 1
            WHEN LOWER(drug) LIKE '%zyvox%' THEN 1
            ELSE 0
        END AS antibiotic
    FROM prescriptions
    -- excludes vials/syringe/normal saline, etc
    WHERE drug_type NOT IN ('BASE')
        -- we exclude routes via the eye, ears, or topically
        AND route NOT IN ('OU', 'OS', 'OD', 'AU', 'AS', 'AD', 'TP')
        AND LOWER(route) NOT LIKE '%ear%'
        AND LOWER(route) NOT LIKE '%eye%'
        -- we exclude certain types of antibiotics: topical creams,
        -- gels, desens, etc
        AND LOWER(drug) NOT LIKE '%cream%'
        AND LOWER(drug) NOT LIKE '%desensitization%'
        AND LOWER(drug) NOT LIKE '%ophth oint%'
        AND LOWER(drug) NOT LIKE '%gel%'
),
vap_patients_temp AS (
    SELECT DISTINCT stay_id, subject_id
    FROM VAP_PATIENTS
)
SELECT
    pr.subject_id, pr.hadm_id, ie.stay_id,
    pr.drug AS antibiotic, pr.route, pr.starttime, pr.stoptime
FROM prescriptions pr
INNER JOIN abx_VAP
    ON pr.drug = abx_VAP.drug
    AND pr.route = abx_VAP.route
LEFT JOIN icustays ie
    ON pr.hadm_id = ie.hadm_id
    AND pr.starttime >= ie.intime
    AND pr.starttime < ie.outtime
WHERE abx_VAP.antibiotic = 1
  AND ie.stay_id IN (SELECT stay_id FROM vap_patients_temp)
"""

conn = sqlite3.connect(db_path)
try:
    antibiotic_data = pd.read_sql_query(query, conn)
    print("Query executed successfully. Here's a preview of the results:")
    print(antibiotic_data.head())

    # Process duration
    antibiotic_data['starttime'] = pd.to_datetime(antibiotic_data['starttime'])
    antibiotic_data['stoptime'] = pd.to_datetime(antibiotic_data['stoptime'])
    antibiotic_data['duration_hours'] = (antibiotic_data['stoptime'] - antibiotic_data['starttime']).dt.total_seconds() / 3600
    antibiotic_data['duration_days'] = antibiotic_data['duration_hours'] / 24

    antibiotic_data.to_sql('abx_VAP', conn, if_exists='replace', index=False)
    print("Table created successfully.")
    
    output_csv_path = '/Users/omaralsuhaibani/Desktop/antibiotics_VAP.csv'
    antibiotic_data.to_csv(output_csv_path, index=False)
    print(f"Results saved to {output_csv_path}")
    
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    conn.close()


# Summarize antibiotic usage
antibiotic_summary = antibiotic_data.groupby(['subject_id', 'antibiotic']).agg(
total_hours=('duration_hours', 'sum')
).reset_index()

# Visualization of top antibiotics by usage
top_antibiotics = antibiotic_summary.groupby('antibiotic')['total_hours'].sum().sort_values(ascending=False).head(10)
plt.bar(top_antibiotics.index, top_antibiotics.values)
plt.title("Top Antibiotics Used in VAP Patients")
plt.xlabel("Antibiotic")
plt.ylabel("Total Hours")
plt.xticks(rotation=90)
plt.show()

Vasoactive Agents:

In [ ]:
query = """
WITH vap_patients_temp AS (
    SELECT DISTINCT stay_id
    FROM VAP_PATIENTS
)

SELECT 
    ie.subject_id,
    ie.stay_id,
    input.itemid,
    CASE 
        WHEN input.itemid = 221662 THEN 'Dopamine'
        WHEN input.itemid = 221289 THEN 'Epinephrine'
        WHEN input.itemid = 221906 THEN 'Norepinephrine'
        WHEN input.itemid = 221749 THEN 'Phenylephrine'
        WHEN input.itemid = 222315 THEN 'Vasopressin'
        ELSE 'Unknown'
    END AS drug_name,
    input.rate AS vaso_rate,
    input.amount AS vaso_amount,
    input.starttime,
    input.endtime
FROM inputevents input
INNER JOIN icustays ie
    ON input.stay_id = ie.stay_id
INNER JOIN vap_patients_temp vap
    ON ie.stay_id = vap.stay_id
WHERE input.itemid IN (
    221662, -- Dopamine
    221289, -- Epinephrine
    221906, -- Norepinephrine
    221749, -- Phenylephrine
    222315 -- Vasopressin
)
"""

conn = sqlite3.connect(db_path)
try:
    vasoactive_results = pd.read_sql_query(query, conn)
    print("Query executed successfully. Here's a preview of the results:")
    print(vasoactive_results.head())

    # Process duration
    vasoactive_results['starttime'] = pd.to_datetime(vasoactive_results['starttime'])
    vasoactive_results['endtime'] = pd.to_datetime(vasoactive_results['endtime'])
    vasoactive_results['duration_hours'] = (vasoactive_results['endtime'] - vasoactive_results['starttime']).dt.total_seconds() / 3600
    vasoactive_results['duration_days'] = vasoactive_results['duration_hours'] / 24

    vasoactive_results.to_sql('vasoactives_VAP', conn, if_exists='replace', index=False)
    print("Table created successfully.")
    
    output_csv_path = '/Users/omaralsuhaibani/Desktop/vasoactives_VAP.csv'
    vasoactive_results.to_csv(output_csv_path, index=False)
    print(f"Results saved to {output_csv_path}")
    
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    conn.close()

**Now to retrieve the top 10 most prescribed antibiotics overall**:

In [ ]:
# Top 10 most prescribed antibiotics overall:
query = """
SELECT
    antibiotic AS drug,
    COUNT(*) AS prescription_count
FROM (
    SELECT antibiotic AS drug_name
    FROM abx_vap
    WHERE hadm_id IN (SELECT hadm_id FROM vap_patients)
    UNION ALL
    SELECT drug_name AS drug_name
    FROM vasoactives_vap
    WHERE stay_id IN (SELECT stay_id FROM vap_patients)
) combined_drugs
GROUP BY drug_name
ORDER BY prescription_count DESC
LIMIT 10;
"""

# Fetch the data
data = pd.read_sql_query(query, conn)

# Horizontal Bar Chart
data.plot(kind='barh', x='drug', y='prescription_count', legend=False, color='teal')
plt.title('Top 10 Most Prescribed Medications for VAP Patients')
plt.xlabel('Prescription Count')
plt.ylabel('Drug')
plt.tight_layout()
plt.show()

Trends in Timing of Medication Administration:

In [ ]:
query = """
SELECT
    STRFTIME('%H', starttime) AS hour_of_day,
    COUNT(*) AS medication_count
FROM prescriptions
WHERE hadm_id IN (SELECT hadm_id FROM vap_patients)
GROUP BY hour_of_day
ORDER BY hour_of_day;
"""
data = pd.read_sql_query(query, conn)

# Ensure no null or unexpected data
data = data.dropna(subset=['hour_of_day'])

# Line Chart
plt.figure(figsize=(10, 6))
plt.plot(data['hour_of_day'], data['medication_count'], marker='o')
plt.title('Trends in Timing of Medication Administration')
plt.xlabel('Hour of Day')
plt.ylabel('Medication Count')
plt.grid()
plt.tight_layout()
plt.show()

**After doing such, we try to find relevent information for VAP patients (retrieve some patients demographics such as gender, dod, admission age, hospital, and ICU stay details):**

In [ ]:
#ICU & hospital stay of VAP patients (and demographics):
query = """
WITH vap_patients_temp AS (
    SELECT DISTINCT stay_id
    FROM VAP_PATIENTS
)
SELECT 
    ie.subject_id, 
    ie.hadm_id, 
    ie.stay_id,

    -- Patient level factors
    pat.gender, 
    pat.dod,

    -- Hospital-level factors
    adm.admittime, 
    adm.dischtime,
    adm.hospital_expire_flag,
    adm.race,
    (julianday(adm.dischtime) - julianday(adm.admittime)) AS los_hospital,

    -- Calculate admission age
    pat.anchor_age + 
    CAST((strftime('%Y', adm.admittime) - pat.anchor_year) AS INTEGER) AS admission_age,
    
    -- ICU-level factors
    ie.intime AS icu_admission, 
    ie.outtime AS icu_discharge,
    ie.los AS los_icu, -- we use the existing 'los' column from the icustays table

     -- Hospital stay sequence
    DENSE_RANK() OVER (
        PARTITION BY adm.subject_id ORDER BY adm.admittime
    ) AS hospstay_seq,

    -- First hospital stay
    CASE
        WHEN DENSE_RANK() OVER (
                PARTITION BY adm.subject_id ORDER BY adm.admittime
            ) = 1 THEN 1 -- True
        ELSE 0 -- False
    END AS first_hosp_stay,

    -- ICU stay sequence
    DENSE_RANK() OVER (
        PARTITION BY ie.hadm_id ORDER BY ie.intime
    ) AS icustay_seq,

    -- First ICU stay for the current hospitalization
    CASE
        WHEN DENSE_RANK() OVER (
                PARTITION BY ie.hadm_id ORDER BY ie.intime
            ) = 1 THEN 1 -- True
        ELSE 0 -- False
    END AS first_icu_stay

FROM icustays ie
INNER JOIN admissions adm
    ON ie.hadm_id = adm.hadm_id
INNER JOIN patients pat
    ON ie.subject_id = pat.subject_id
WHERE ie.stay_id IN (SELECT stay_id FROM vap_patients_temp); -- Filter for VAP patients
"""

conn = sqlite3.connect(db_path)
try:
    VAPPatientsstay_results = pd.read_sql_query(query, conn)
    print("Query executed successfully. Here's a preview of the results:")
    print(VAPPatientsstay_results.head())

    VAPPatientsstay_results.to_sql('VAPPatientsStay', conn, if_exists='replace', index=False)
    print("Table created successfully.")
    
    output_csv_path = '//VAPPatientsStay.csv'
    VAPPatientsstay_results.to_csv(output_csv_path, index=False)
    print(f"Results saved to {output_csv_path}")

except Exception as e:
    print(f"An error occurred: {e}")

**Now, we do santiy checks to validate that the information is correct, and that all VAP patients have an ICU admission**: 

In [ ]:
# ICU details
query = """
WITH admissions_with_icu AS (
    SELECT DISTINCT hadm_id, stay_id
    FROM icustays
),
vap_admissions AS (
    SELECT DISTINCT v.hadm_id
    FROM VAP_PATIENTS v
),
vap_without_icu AS (
    SELECT va.hadm_id
    FROM vap_admissions va
    LEFT JOIN admissions_with_icu ai
    ON va.hadm_id = ai.hadm_id
    WHERE ai.stay_id IS NULL
)
SELECT COUNT(*) AS no_icu_count, (SELECT COUNT(*) FROM vap_admissions) AS total_vap_count
FROM vap_without_icu;
"""
try:
    VAPPatientsstay_results = pd.read_sql_query(query, conn)
    print("Query executed successfully. Here's a preview of the results:")
    print(VAPPatientsstay_results.head())

    VAPPatientsstay_results.to_sql('VAPPatientsStay', conn, if_exists='replace', index=False)
    print("Table created successfully.")
    
    output_csv_path = '//VAPPatientsStay.csv'
    VAPPatientsstay_results.to_csv(output_csv_path, index=False)
    print(f"Results saved to {output_csv_path}")

except Exception as e:
    print(f"An error occurred: {e}")

Merge ICU stays with VAP admissions:

In [ ]:
vap_admissions = admissions.merge(vap_patients[['hadm_id']].drop_duplicates(), on='hadm_id', how='inner')
icu_admissions = icustays[['hadm_id']].drop_duplicates()
vap_without_icu = vap_admissions[~vap_admissions['hadm_id'].isin(icu_admissions['hadm_id'])]

plt.bar(['With ICU', 'Without ICU'], [len(vap_admissions) - len(vap_without_icu), len(vap_without_icu)])
plt.title('VAP Admissions With and Without ICU Stays')
plt.ylabel('Count')
plt.show()

Different visualizations and queries for sanity checks and demographics:

In [ ]:
#Age Distribution of VAP Patients
query = """
SELECT
    v.admission_age AS age,
    COUNT(*) AS patient_count
FROM VAP_PATIENTS v
GROUP BY v.admission_age
ORDER BY v.admission_age;
"""

data = pd.read_sql_query(query, conn)
# Plotting the histogram
plt.figure(figsize=(10, 6))
plt.bar(data['age'], data['patient_count'], color='skyblue', edgecolor='black')
plt.title('Age Distribution of VAP Patients', fontsize=14)
plt.xlabel('Age', fontsize=12)
plt.ylabel('Number of Patients', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

#VAP Admissions Without ICU Stays
query = """
SELECT
    CASE
        WHEN icu.stay_id IS NULL THEN 'Without ICU Stay'
        ELSE 'With ICU Stay'
    END AS admission_status,
    COUNT(DISTINCT v.hadm_id) AS admission_count
FROM VAP_PATIENTS v
LEFT JOIN icustays icu
    ON v.hadm_id = icu.hadm_id
GROUP BY admission_status;
"""
data = pd.read_sql_query(query, conn)
# Bar Chart
plt.bar(data['admission_status'], data['admission_count'], color=['red', 'green'])
plt.title('VAP Admissions With and Without ICU Stays')
plt.ylabel('Count of Admissions')
plt.xlabel('Admission Status')
plt.tight_layout()
plt.show()

#VAP Patients Admitted to ICU at Any Point
query = """
SELECT
    CASE
        WHEN icu.hadm_id IS NULL THEN 'Non-ICU Admissions Only'
        ELSE 'At Least One ICU Admission'
    END AS icu_admission_status,
    COUNT(DISTINCT v.subject_id) AS patient_count
FROM VAP_PATIENTS v
LEFT JOIN icustays icu
    ON v.hadm_id = icu.hadm_id
GROUP BY icu_admission_status;
"""
data = pd.read_sql_query(query, conn)
# Bar Chart
plt.bar(data['icu_admission_status'], data['patient_count'], color=['orange', 'blue'])
plt.title('ICU vs. Non-ICU Admissions for VAP Patients')
plt.ylabel('Count of Patients')
plt.xlabel('ICU Admission Status')
plt.tight_layout()
plt.show()

#Medications Administered Outside ICU
query = """
SELECT
    CASE
        WHEN icu.hadm_id IS NOT NULL THEN 'ICU'
        ELSE 'Non-ICU'
    END AS location,
    COUNT(p.drug) AS medication_count
FROM prescriptions p
LEFT JOIN icustays icu
    ON p.hadm_id = icu.hadm_id
WHERE p.hadm_id IN (
    SELECT hadm_id FROM VAP_PATIENTS
)
GROUP BY location;
"""
data = pd.read_sql_query(query, conn)
# Grouped Bar Chart
plt.bar(data['location'], data['medication_count'], color=['blue', 'red'])
plt.title('Medications Administered: ICU vs. Non-ICU')
plt.ylabel('Count of Medications')
plt.xlabel('Location')
plt.tight_layout()
plt.show()

# Gender distribution
query = """
SELECT
    p.gender,
    COUNT(*) AS count
FROM vap_patients vp
JOIN patients p ON vp.subject_id = p.subject_id
GROUP BY p.gender;
"""
gender_data = pd.read_sql_query(query, conn)
# Bar Graph
plt.figure(figsize=(6, 6))
plt.bar(gender_data['gender'], gender_data['count'], color=['salmon', 'skyblue'])
plt.title('Gender Distribution of VAP Patients')
plt.xlabel('Gender')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

Monthly Trends in VAP Admissions:

In [ ]:
query = """
SELECT
    STRFTIME('%m', a.admittime) AS month,
    COUNT(*) AS admission_count
FROM vap_patients v
JOIN admissions a
    ON v.hadm_id = a.hadm_id
GROUP BY month
ORDER BY month;
"""
data = pd.read_sql_query(query, conn)

# Convert month numbers to month names for better readability
data['month'] = data['month'].apply(lambda x: pd.to_datetime(x, format='%m').strftime('%B'))
# Line Graph
plt.figure(figsize=(10, 6))
plt.plot(data['month'], data['admission_count'], marker='o')
plt.title('Monthly Trends in VAP Admissions', fontsize=14)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Admission Count', fontsize=12)
plt.grid()
plt.tight_layout()
plt.xticks(rotation=45)
plt.show()

Descriptive statistics to analyze ICU and Hospital Length of Stay (utilizing gender and ages of VAP patients):


In [ ]:
# Group by the gender and length of stay in ICU
gender_icu_los_stats = VAPPatientsstay_results.groupby('gender')['los_icu'].describe()
print(gender_icu_los_stats)

# ICU Length of Stay by Gender
plt.figure(figsize=(10, 6))
sns.boxplot(data=VAPPatientsstay_results, x='gender', y='los_icu')
plt.title('ICU Length of Stay (LOS) by Gender for VAP Patients')
plt.xlabel('Gender')
plt.ylabel('ICU Length of Stay (days)')
plt.show()

# Group by the gender and length of stay in Hospital
gender_hospital_los_stats = VAPPatientsstay_results.groupby('gender')['los_hospital'].describe()
print(gender_hospital_los_stats)

# Hospital Length of Stay by Gender
plt.figure(figsize=(10, 6))
sns.boxplot(data=VAPPatientsstay_results, x='gender', y='los_hospital')
plt.title('Hospital Length of Stay (LOS) by Gender for VAP Patients')
plt.xlabel('Gender')
plt.ylabel('Hospital Length of Stay (days)')
plt.show()

# Column for age groups and group by age group and length of stay in ICU
bins = [18, 19, 30, 40, 50, 60, 70, 80, 90, 100]
labels = ['18-25', '26-35', '36-45', '46-54', '55-64', '65-74', '75-84', '85-94', '94-100']
VAPPatientsstay_results['age_group'] = pd.cut(VAPPatientsstay_results['admission_age'], bins=bins, labels=labels, right=False)
admission_age_icu_los_stats = VAPPatientsstay_results.groupby('age_group')['los_icu'].describe()
admission_age_hosp_los_stats = VAPPatientsstay_results.groupby('age_group')['los_hospital'].describe()
print(admission_age_icu_los_stats)
print(admission_age_hosp_los_stats)

# ICU Length of Stay by Age Group
plt.figure(figsize=(20, 6))
sns.boxplot(data=VAPPatientsstay_results, x='age_group', y='los_icu')
plt.title('ICU Length of Stay (LOS) by Age Group for VAP Patients')
plt.xlabel('Age Group')
plt.ylabel('ICU Length of Stay (days)')
plt.show()

# Hospital Length of Stay by Age Group
plt.figure(figsize=(20, 6))
sns.boxplot(data=VAPPatientsstay_results, x='age_group', y='los_hospital')
plt.title('Hospital Length of Stay (LOS) by Age Group for VAP Patients')
plt.xlabel('Age Group')
plt.ylabel('Hospital Length of Stay (days)')
plt.show()

**After doing so, we try to find relevency by analyzing the patient's admission locationt**:

In [ ]:
#Display the unique admissions locations for review
query = """
SELECT DISTINCT admission_location
FROM admissions a
JOIN VAP_PATIENTS vp
ON a.hadm_id = vp.hadm_id;
"""

try:
    admission_locations = pd.read_sql_query(query, conn)
    print("Query executed successfully. Here's a preview of the unique admission locations:")
    print(admission_locations.head(30))

except Exception as e:
    print(f"An error occurred: {e}")

Execute query to get top 10 most prescribed medications:

In [ ]:
# Execute query for top 10 antibiotic classes
query = """
SELECT
    antibiotic AS antibiotic_class,
    COUNT(*) AS usage_count
FROM abx_vap
GROUP BY antibiotic_class
ORDER BY usage_count DESC
LIMIT 10;
"""
data = pd.read_sql_query(query, conn)

# Ensure no null data
data = data.dropna(subset=['antibiotic_class'])

# Vertical Bar Chart for Top 5 Antibiotic Classes
plt.figure(figsize=(12, 6))
plt.bar(data['antibiotic_class'], data['usage_count'], color='skyblue')
plt.title('Top 10 Antibiotics by Class for VAP Patients')
plt.xlabel('Antibiotic Class')
plt.ylabel('Usage Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

Trends in timing of medication administration:

In [ ]:
query = """
SELECT
    STRFTIME('%H', starttime) AS hour_of_day,
    COUNT(*) AS medication_count
FROM prescriptions
WHERE hadm_id IN (SELECT hadm_id FROM vap_patients)
GROUP BY hour_of_day
ORDER BY hour_of_day;
"""
data = pd.read_sql_query(query, conn)

# Ensure no null or unexpected data
data = data.dropna(subset=['hour_of_day'])

# Line Chart
plt.figure(figsize=(10, 6))
plt.plot(data['hour_of_day'], data['medication_count'], marker='o')
plt.title('Trends in Timing of Medication Administration')
plt.xlabel('Hour of Day')
plt.ylabel('Medication Count')
plt.grid()
plt.tight_layout()
plt.show()

Top 5 Most Frequent Administration Routes for Medications:

In [ ]:
query = """
SELECT
    route,
    COUNT(*) AS frequency
FROM prescriptions
WHERE hadm_id IN (SELECT hadm_id FROM vap_patients)
GROUP BY route
ORDER BY frequency DESC
LIMIT 5;
"""
data = pd.read_sql_query(query, conn)

# Handle None values
data = data.dropna(subset=['route'])

# Horizontal Bar Chart for Top 5 Administration Routes
plt.figure(figsize=(10, 5))
plt.barh(data['route'], data['frequency'], color='skyblue', edgecolor='black')
plt.title('Top 5 Most Frequent Administration Routes for Medications')
plt.xlabel('Frequency')
plt.ylabel('Route')
plt.tight_layout()
plt.show()

Get mortality and age data:

In [ ]:
query = """
SELECT
    vp.admission_age AS age,
    a.hospital_expire_flag AS mortality
FROM vap_patients vp
JOIN admissions a ON vp.hadm_id = a.hadm_id;
"""

mortality_data = pd.read_sql_query(query, conn)

# Boxplot
plt.figure(figsize=(8, 6))
plt.boxplot([mortality_data[mortality_data['mortality'] == 0]['age'], 
             mortality_data[mortality_data['mortality'] == 1]['age']],
            labels=['Survived', 'Not Survived'], patch_artist=True)
plt.title('Boxplot of Age by Mortality')
plt.ylabel('Age')
plt.xlabel('Mortality Status')
plt.tight_layout()
plt.show()

Time to death for deceased patients:

In [ ]:
query = """
SELECT
    p.anchor_age AS age,
    JULIANDAY(a.deathtime) - JULIANDAY(a.admittime) AS time_to_death
FROM vap_patients vp
JOIN patients p ON vp.subject_id = p.subject_id
JOIN admissions a ON vp.hadm_id = a.hadm_id
WHERE a.hospital_expire_flag = 1 AND a.deathtime IS NOT NULL;
"""

# Load data
time_to_death_data = pd.read_sql_query(query, conn)

# Histogram
colors = cm.viridis(np.linspace(0, 1, len(time_to_death_data["time_to_death"])))
plt.figure(figsize=(10, 6))
plt.hist(time_to_death_data['time_to_death'], bins=20, edgecolor='black', alpha=0.7)
plt.title('Histogram of Time to Death')
plt.xlabel('Time to Death (Days)')
plt.ylabel('Frequency')
plt.grid(axis='y')
plt.tight_layout()
plt.show()

ICU length of stay for VAP patients (in days):

In [ ]:
query = """
SELECT
    icu.hadm_id,
    ROUND((JULIANDAY(icu.outtime) - JULIANDAY(icu.intime)), 2) AS icu_los_days
FROM icustays icu
WHERE icu.hadm_id IN (SELECT hadm_id FROM vap_patients)
"""

icu_data = pd.read_sql_query(query, conn)

# Ensuring the data integrity by dropping rows with null or negative values for ICU los
icu_data = icu_data.dropna(subset=["icu_los_days"])
icu_data = icu_data[icu_data["icu_los_days"] > 0]  # Filter invalid data

# Check data preview
print(icu_data.head())

# Visualization 1: Box Plot
plt.figure(figsize=(10, 6))
plt.boxplot(icu_data["icu_los_days"], vert=False, patch_artist=True)
plt.title("ICU Length of Stay for VAP Patients", fontsize=16)
plt.xlabel("ICU Length of Stay (days)", fontsize=14)
plt.tight_layout()
plt.show()

# Visualization 2: Histogram
plt.figure(figsize=(10, 6))
plt.hist(icu_data["icu_los_days"], bins=20, edgecolor="black", alpha=0.7)
plt.title("Distribution of ICU Length of Stay (in Days) for VAP Patients", fontsize=16)
plt.xlabel("ICU Length of Stay (days)", fontsize=14)
plt.ylabel("Number of Patients", fontsize=14)
plt.tight_layout()
plt.show()

Trends in comorbidities among VAP patients:

In [ ]:
query = """
WITH diag AS (
    SELECT
        hadm_id,
        CASE WHEN icd_version = 9 THEN icd_code ELSE NULL END AS icd9_code,
        CASE WHEN icd_version = 10 THEN icd_code ELSE NULL END AS icd10_code
    FROM diagnoses_icd
),
com AS (
    SELECT
        vp.hadm_id,

        -- Myocardial infarction
        MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) IN ('410', '412') OR
            SUBSTR(icd10_code, 1, 3) IN ('I21', 'I22') OR
            SUBSTR(icd10_code, 1, 4) = 'I252'
            THEN 1 ELSE 0 END) AS myocardial_infarct,

        -- Congestive heart failure
        MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) = '428' OR
            SUBSTR(icd9_code, 1, 5) IN ('39891', '40201', '40211', '40291', '40401', '40403',
                                        '40411', '40413', '40491', '40493') OR
            SUBSTR(icd9_code, 1, 4) BETWEEN '4254' AND '4259' OR
            SUBSTR(icd10_code, 1, 3) IN ('I43', 'I50') OR
            SUBSTR(icd10_code, 1, 4) IN ('I099', 'I110', 'I130', 'I132', 'I255', 'I420', 
                                         'I425', 'I426', 'I427', 'I428', 'I429', 'P290')
            THEN 1 ELSE 0 END) AS congestive_heart_failure,

        -- Peripheral vascular disease
        MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) IN ('440', '441') OR
            SUBSTR(icd9_code, 1, 4) IN ('0930', '4373', '4471', '5571', '5579', 'V434') OR
            SUBSTR(icd9_code, 1, 4) BETWEEN '4431' AND '4439' OR
            SUBSTR(icd10_code, 1, 3) IN ('I70', 'I71') OR
            SUBSTR(icd10_code, 1, 4) IN ('I731', 'I738', 'I739', 'I771', 'I790', 'I792', 
                                         'K551', 'K558', 'K559', 'Z958', 'Z959')
            THEN 1 ELSE 0 END) AS peripheral_vascular_disease,

        -- Cerebrovascular disease
        MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) BETWEEN '430' AND '438' OR
            SUBSTR(icd9_code, 1, 5) = '36234' OR
            SUBSTR(icd10_code, 1, 3) IN ('G45', 'G46') OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'I60' AND 'I69' OR
            SUBSTR(icd10_code, 1, 4) = 'H340'
            THEN 1 ELSE 0 END) AS cerebrovascular_disease,

        -- Chronic pulmonary disease
        MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) BETWEEN '490' AND '505' OR
            SUBSTR(icd9_code, 1, 4) IN ('4168', '4169', '5064', '5081', '5088') OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'J40' AND 'J47' OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'J60' AND 'J67' OR
            SUBSTR(icd10_code, 1, 4) IN ('I278', 'I279', 'J684', 'J701', 'J703')
            THEN 1 ELSE 0 END) AS chronic_pulmonary_disease,

        -- Diabetes without chronic complication
        MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 4) IN ('2500', '2501', '2502', '2503', '2508', '2509') OR
            SUBSTR(icd10_code, 1, 4) IN ('E100', 'E101', 'E106', 'E108', 'E109', 'E110', 
                                         'E111', 'E116', 'E118', 'E119', 'E120', 'E121', 
                                         'E126', 'E128', 'E129', 'E130', 'E131', 'E136', 
                                         'E138', 'E139', 'E140', 'E141', 'E146', 'E148', 'E149')
            THEN 1 ELSE 0 END) AS diabetes_without_cc,

        -- Diabetes with chronic complication
        MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 4) IN ('2504', '2505', '2506', '2507') OR
            SUBSTR(icd10_code, 1, 4) IN ('E102', 'E103', 'E104', 'E105', 'E107', 'E112', 
                                         'E113', 'E114', 'E115', 'E117', 'E122', 'E123', 
                                         'E124', 'E125', 'E127', 'E132', 'E133', 'E134', 
                                         'E135', 'E137', 'E142', 'E143', 'E144', 'E145', 'E147')
            THEN 1 ELSE 0 END) AS diabetes_with_cc,

        -- Renal disease
        MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) IN ('582', '585', '586', 'V56') OR
            SUBSTR(icd10_code, 1, 3) IN ('N18', 'N19') OR
            SUBSTR(icd10_code, 1, 4) IN ('N032', 'N033', 'N034', 'N035', 'N036', 'N037', 
                                         'N052', 'N053', 'N054', 'N055', 'N056', 'N057')
            THEN 1 ELSE 0 END) AS renal_disease,

        -- Liver disease
        MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) IN ('570', '571') OR
            SUBSTR(icd9_code, 1, 5) IN ('5722', '5723', '5724', '5728') OR
            SUBSTR(icd10_code, 1, 3) IN ('K70', 'K71', 'K72', 'K73', 'K74') OR
            SUBSTR(icd10_code, 1, 4) IN ('K760', 'K761', 'K762', 'K763', 'K764', 'K768', 'K769')
            THEN 1 ELSE 0 END) AS liver_disease,

        -- Cancer
        MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) BETWEEN '140' AND '172' OR
            SUBSTR(icd9_code, 1, 3) BETWEEN '174' AND '195' OR
            SUBSTR(icd10_code, 1, 3) IN ('C00', 'C01', 'C02', 'C03', 'C04', 'C05', 'C06', 'C07', 
                                         'C08', 'C09', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16')
            THEN 1 ELSE 0 END) AS cancer
    FROM vap_patients vp
    LEFT JOIN diag
        ON vp.hadm_id = diag.hadm_id
    GROUP BY vp.hadm_id
)

SELECT
    SUM(myocardial_infarct) AS myocardial_infarct,
    SUM(congestive_heart_failure) AS congestive_heart_failure,
    SUM(peripheral_vascular_disease) AS peripheral_vascular_disease,
    SUM(cerebrovascular_disease) AS cerebrovascular_disease,
    SUM(chronic_pulmonary_disease) AS chronic_pulmonary_disease,
    SUM(diabetes_without_cc) AS diabetes_without_cc,
    SUM(diabetes_with_cc) AS diabetes_with_cc,
    SUM(renal_disease) AS renal_disease,
    SUM(liver_disease) AS liver_disease,
    SUM(cancer) AS cancer
FROM com;
"""

# Execute query
data = pd.read_sql_query(query, conn)

# Reshape data for visualization
data_melted = data.melt(var_name="Comorbidity", value_name="Count")

# Generate a colormap
colors = cm.viridis(np.linspace(0, 1, len(data_melted["Comorbidity"].unique())))

# Bar chart with multicolor bars
plt.figure(figsize=(12, 6))
plt.bar(data_melted["Comorbidity"], data_melted["Count"], color=colors, edgecolor="black")
plt.title("Comorbidity Trends Among VAP Patients", fontsize=14)
plt.xlabel("Comorbidity", fontsize=12)
plt.ylabel("Count", fontsize=12)
plt.xticks(rotation=90, ha="right", fontsize=10)
plt.tight_layout()
plt.show()